In [1]:
# Header Block. You should NOT rerun this block if you have not restarted the kernel.
from apercal import lib
lib.setup_logger('debug', logfile='/home/frank/ngc4278-tutorial.log')
import pylab as pl
%matplotlib inline
import os
from apercal import calibrate

root - INFO : Logging started!
root - INFO : To see the log in a bash window use the following command:
root - INFO : tail -n +1 -f /home/frank/ngc4278-tutorial.log


Setup logger with lib.setup_logger()


In [2]:
ccal = calibrate.crosscal()

In [34]:
cal1 = calibrate.source()
path = '/home/frank/tutorial-ngc4278'
cal1.path = path
cal1.ms = '10400803_S0_T0.MS'
cal1.vis = 'cal1.uv'
cal1.update()
print cal1.vis

cal2 = calibrate.source()
cal2.path = path
cal2.ms = '10400805_S0_T0.MS'
cal2.vis = 'cal2.uv'
cal2.update()

src1 = calibrate.source()
src1.path = path
src1.ms = '10400804_S0_T0.MS'
src1.vis = 'src1.uv'
src1.update()

src2 = calibrate.source()
src2.path = path
src2.ms = '10400804_S0_T1.MS'
src2.vis = 'src2.uv'
src2.update()

src3 = calibrate.source()
src3.path = path
src3.ms = '10400804_S0_T2.MS'
src3.vis = 'src3.uv'
src3.update()

src = calibrate.source()
src.path = path
src.vis = 'src.uv'
src.update()

s0 = calibrate.source()
s0.path = path
s0.vis = 'src.uv_chan0'
s0.update()


cal1.uv


In [13]:
# First, define the source that you want to import
source = src2 # Just change this one.

# Send this to the crosscal object. 

ccal.source = source

# Now we move into the working directory
ccal.setup()

ccal.ms2uvfits()


crosscal - WARNING : You have now moved into /home/frank/tutorial-ngc4278
crosscal - WARNING : All outputs will be relative to this path.
ms2uvfits - INFO : ms2uvfits: Converting MS to UVFITS Format
ms2uvfits - INFO : MS: 10400804_S0_T1.MS
ms2uvfits - INFO : UVFITS: 10400804_S0_T1.UVF
ms2uvfits - INFO : Directory: 
masher - DEBUG : ms2uvfits multisource=T writesyscal=T fitsfile=10400804_S0_T1.UVF ms=10400804_S0_T1.MS combinespw=T 
basher - DEBUG : ms2uvfits multisource=T writesyscal=T fitsfile=10400804_S0_T1.UVF ms=10400804_S0_T1.MS combinespw=T 
basher - DEBUG : Command = ms2uvfits multisource=T writesyscal=T fitsfile=10400804_S0_T1.UVF ms=10400804_S0_T1.MS combinespw=T 
basher - DEBUG : 
ms2uvfits: Version 20080124APS
The input MeasurementSet is: /home/frank/tutorial-ngc4278/10400804_S0_T1.MS
ms2uvfits normally ended

basher - ERROR : 2015-10-07 13:48:48	INFO	MSFitsOutputAstron::writeFitsFile	Converting MeasurementSet /home/frank/tutorial-ngc4278/10400804_S0_T1.MS to FITS file '10400

In [20]:
prthd = lib.miriad('prthd')
prthd.in_ = cal1.vis
output = prthd.go()

masher - DEBUG : prthd in_=cal1.uv 
basher - DEBUG : prthd in_=cal1.uv 
basher - DEBUG : Command = prthd in_=cal1.uv 
basher - DEBUG : 
Prthd: version 11-jul-07
****************************************************************
Filename: cal1.uv
Telescope: WSRT
Object: 3c147               Observer: ProcessDZB v6.0
First time: 04FEB03:20:20:39.6
Number of antennae: 14
Polarisations Present: XX,YY
Type of correlations present: mixed-auto-cross
----------------------------------------------------------------
Spectral Correlations:
  Spectrum  Channels  Freq(chan=1)  Increment  Restfreq
      1       1024       1.42761    -0.000020   1.42041 GHz
  Total number of correlations: 6451200
  Correlations are stored in 32-bit form
----------------------------------------------------------------
J2000    Source RA: 05:42:36.138  Dec: +49:51:07.23
Apparent Source RA: 05:42:55.601  Dec: +49:51:26.59

History item is present

basher - DEBUG : Returning output.
miriad prthd - INFO : Completed.


In [36]:
# First, define the source that you want to import
source = src3 # Just change this one.

# Send this to the crosscal object. 

ccal.source = source

# Now we move into the working directory
ccal.setup()

# Setup the WSRTFITS parameters. The inp() method simply shows you the variables that have already been set. 
ccal.wsrtfits.inp()
ccal.wsrtfits.in_ = source.uvf
ccal.wsrtfits.out = source.vis+'.temp'
# The go() method always returns the task output as a list of text.
output = ccal.wsrtfits.go(rmfiles=True)

# Do
ccal.attsys.vis = source.vis+'.temp'
ccal.attsys.out = source.vis
ccal.attsys.go(rmfiles=True)

# Replace the old vis with this new file
#lib.basher('mv '+source.vis+'.temp '+source.vis)
lib.basher('rm -r '+source.vis+'.temp')
#ccal.puthd()
ccal.puthd.in_ = source.vis+'/restfreq'
ccal.puthd.value ='1.420405752'
ccal.puthd.go()
ccal.puthd.in_ = source.vis+'/interval'
ccal.puthd.type = 'double'
ccal.puthd.value= '1.0'
ccal.puthd.go()

#ccal.uvflag()
# Flag antenna 6, autocorrelations and for shadowing
select = ['auto', 'shadow(25)']
ccal.uvflag.vis = source.vis
ccal.uvflag.flagval = 'flag'
for s in select:
    ccal.uvflag.select = s
    ccal.uvflag.go()


crosscal - WARNING : You have now moved into /home/frank/tutorial-ngc4278
crosscal - WARNING : All outputs will be relative to this path.
miriad wsrtfits - INFO : velocity=optbary, task=wsrtfits, out=src2.uv.temp, in_=10400804_S0_T1.UVF, op=uvin
miriad wsrtfits - DEBUG : Cleanup - files will be DELETED.
masher - DEBUG : wsrtfits velocity=optbary in_=10400804_S0_T2.UVF op=uvin out=src3.uv.temp 
basher - DEBUG : wsrtfits velocity=optbary in_=10400804_S0_T2.UVF op=uvin out=src3.uv.temp 
basher - DEBUG : Command = wsrtfits velocity=optbary in_=10400804_S0_T2.UVF op=uvin out=src3.uv.temp 
basher - DEBUG : 
wsrtfits: version 1.3.1 21-Jul-04
 10400804_S0_T2.UVF                                                                                                              
Assuming systemp=30
Assuming jyperk=8
Reading AIPS TY table
Reading AIPS AN table
 freqref    1417588776.8839998     
Decrementing times for configration  1 by  0.40 seconds (UTC-UT1).
 Mount type :           1
Reading AIPS FQ 

In [39]:
# MFCAL
ccal.mfcal.vis = cal1.vis
ccal.mfcal.interval = 10000
ccal.mfcal.edge = '10,50'
ccal.mfcal.refant = 2
output = ccal.mfcal.go()

masher - DEBUG : mfcal vis=cal1.uv edge=10,50 refant=2 interval=10000 
basher - DEBUG : mfcal vis=cal1.uv edge=10,50 refant=2 interval=10000 
basher - DEBUG : Command = mfcal vis=cal1.uv edge=10,50 refant=2 interval=10000 
basher - DEBUG : 
MfCal: version 1.1 11-jan-10
Selecting parallel-hand polarisations
Reading the data ...
Number correlations accepted: 5263440
Number of frequency bands/settings: 1
Number of polarisations selected: 2
Number of solution intervals: 1
Number of passband solution intervals: 1
Initialising ...
Using known frequency variation of 3c147
Generating initial solution estimate ...
Doing solution refinement ...
Iter= 1, Solution Error:  0.207
Iter= 2, Solution Error:  0.000
Saving solution ...

basher - ERROR : ### Warning [mfcal]:  Correlations flagged or edge-rejected: 327600

basher - DEBUG : Returning output.
miriad mfcal - INFO : Completed.


In [37]:
# UVCAT / UVCAL

ccal.source = src
ccal.setup()

ccal.uvcat.vis = src1.vis+','+src2.vis+','+src3.vis
ccal.uvcat.out = src.vis
output = ccal.uvcat.go()

crosscal - WARNING : You have now moved into /home/frank/tutorial-ngc4278
crosscal - WARNING : All outputs will be relative to this path.
masher - DEBUG : uvcat vis=src1.uv,src2.uv,src3.uv out=src.uv 
basher - DEBUG : uvcat vis=src1.uv,src2.uv,src3.uv out=src.uv 
basher - DEBUG : Command = uvcat vis=src1.uv,src2.uv,src3.uv out=src.uv 
basher - DEBUG : 
UvCat: version 6-jun-2012

basher - DEBUG : Returning output.
miriad uvcat - INFO : Completed.


In [40]:
# GPCOPY
ccal.gpcopy.vis = cal1.vis
ccal.gpcopy.out = src.vis
output = ccal.gpcopy.go()

masher - DEBUG : gpcopy vis=cal1.uv out=src.uv 
basher - DEBUG : gpcopy vis=cal1.uv out=src.uv 
basher - DEBUG : Command = gpcopy vis=cal1.uv out=src.uv 
basher - DEBUG : 
GpCopy: version 23-Jan-07
Copying gain table
Interpolation tolerance set to (minutes): 720.00
Copying bandpass table

basher - DEBUG : Returning output.
miriad gpcopy - INFO : Completed.


In [41]:
# UVLIN
ccal.uvlin.vis = src.vis
ccal.uvlin.chans = '50,300,400,600,601,800'
ccal.uvlin.order = 2
ccal.uvlin.mode = 'chan0'
ccal.uvlin.out = s0.vis
output = ccal.uvlin.go()

masher - DEBUG : uvlin vis=src.uv out=src.uv_chan0 mode=chan0 chans=50,300,400,600,601,800 order=2 
basher - DEBUG : uvlin vis=src.uv out=src.uv_chan0 mode=chan0 chans=50,300,400,600,601,800 order=2 
basher - DEBUG : Command = uvlin vis=src.uv out=src.uv_chan0 mode=chan0 chans=50,300,400,600,601,800 order=2 
basher - DEBUG : 
UvLin: version 1.0 24-Jan-07
Applying bandpass corrections to src.uv
Applying gain corrections to src.uv
Visibilities where continuum fit failed: 30842
Visibilities where 2nd  order fit used: 119938

basher - DEBUG : Returning output.
miriad uvlin - INFO : Completed.


## Deep Image

In [42]:
scal = calibrate.wselfcal()

In [44]:
# scal.invert is attached to the invert task.
# Defaults for the map and beam parameters are set in the next block
# Change parameters as follows:

scal.source = s0
scal.output = 'output'

scal.invert.robust = 0.0
scal.invert.slop = '0.5'
scal.invert.imsize ='1500'
scal.invert.cell = 4
scal.invert.options='mfs,double'
scal.invert.select = '-uvrange(0,0.5)'

# scal.clean is attached to the clean task 
# Defaults for the map, beam and model fields are set in the next block
# Change parameters as follows:
scal.clean.cutoff = 5e-4
scal.clean.niters=100000

# scal.maths is attached to the MATHS task
# Parameters are shown here for illustration. 
# Change parameters as follows:
scal.maths = lib.miriad('maths')
scal.maths.exp = 'map'
scal.maths.mask = 'map.gt.1e-3'
scal.maths.out = 'mask'

# scal.restor is attached to the restor task
# Change parameters as follows:
scal.restor.mode = 'clean'

# scal.imstat is attached to the imstat task

# SelfCal and Image Cycle Settings
# Number of Major Cycles
scal.num_major = 3
# Number of minor cycles - The number of image cycles
scal.num_minor = 3

# The mask cutoffs will go from immax/cmin...
scal.cmin = 3
# to a minimum of imax/cmax
scal.cmax = 10
# If linear is True, then the mask cutoffs will decrease linearly.
# If linear is False, then the mask cutoffs will be decrease logarithmically, 
# with log10(cmin) and log10(cmax) as the minimum and maximum denominators respectively.
scal.linear = True 
# This is how much deeper than the mask cutoff that we will clean to.
scal.d = 10.
# Cleaning and masking will stop at 7*theoretical rms 
scal.nsigma = 7 
# dummy value for the image maximum
scal.immax = 0.0
scal.rmgains = False # Remove gains from vis file.

scal.setup()

scal.deep_image()

selfcal - WARNING : Cannot find /home/frank/tutorial-ngc4278/output, making it 
selfcal - WARNING : You have now moved into /home/frank/tutorial-ngc4278/output
selfcal - WARNING : All outputs will be relative to this path.
selfcal - INFO : Making symbolic/soft link to your visibility file
basher - DEBUG : ln -s /home/frank/tutorial-ngc4278/src.uv_chan0
basher - DEBUG : Returning output.
masher - DEBUG : obsrms jyperk=150 tsys=50 inttime=720.0 bw=20 coreta=0.88 theta=15 freq=1.4 antdiam=25 nants=11 
basher - DEBUG : obsrms jyperk=150 tsys=50 inttime=720.0 bw=20 coreta=0.88 theta=15 freq=1.4 antdiam=25 nants=11 
basher - DEBUG : Command = obsrms jyperk=150 tsys=50 inttime=720.0 bw=20 coreta=0.88 theta=15 freq=1.4 antdiam=25 nants=11 
basher - DEBUG : 
OBSRMS: version 28-SEP-2007
 tsys  jyperk  freq   lambda    deltav     bw  inttime   nants    theta    coreta   rmsphase
   50.   9.4    1.40   214.14  4282.75   20.00  720.00    11.   15.0   15.0  0.88    0.
Rms Flux density:   0.546E-01 m

## SelfCal

In [47]:
# scal.invert is attached to the invert task.
# Defaults for the map and beam parameters are set in the next block
# Change parameters as follows:

scal.source = s0
scal.output = 'selfcal_3'

scal.selfcal.select = '-uvrange(0,0.5)'
scal.selfcal.options = 'mfs,phase'
scal.selfcal.refant = '2'
scal.selfcal.interval = 2
scal.selfcal.clip = 1e-6

# SelfCal and Image Cycle Settings
# Number of Major Cycles
scal.num_major = 4
# Number of minor cycles - The number of image cycles
scal.num_minor = 5

# The mask cutoffs will go from immax/cmin...
scal.cmin = 3
# to a minimum of imax/cmax
scal.cmax = 15
# If linear is True, then the mask cutoffs will decrease linearly.
# If linear is False, then the mask cutoffs will be decrease logarithmically, 
# with log10(cmin) and log10(cmax) as the minimum and maximum denominators respectively.
scal.linear = True 
# This is how much deeper than the mask cutoff that we will clean to.
scal.d = 10.
# Cleaning and masking will stop at 7*theoretical rms 
scal.nsigma = 7 
# dummy value for the image maximum
scal.immax = 0.0
scal.rmgains = False # Remove gains from vis file.

scal.setup()

scal.go()

selfcal - WARNING : Cannot find /home/frank/tutorial-ngc4278/selfcal_3, making it 
selfcal - WARNING : You have now moved into /home/frank/tutorial-ngc4278/selfcal_3
selfcal - WARNING : All outputs will be relative to this path.
selfcal - INFO : Making symbolic/soft link to your visibility file
basher - DEBUG : ln -s /home/frank/tutorial-ngc4278/src.uv_chan0
basher - DEBUG : Returning output.
masher - DEBUG : obsrms jyperk=150 tsys=50 inttime=720.0 bw=20 coreta=0.88 theta=15 freq=1.4 antdiam=25 nants=11 
basher - DEBUG : obsrms jyperk=150 tsys=50 inttime=720.0 bw=20 coreta=0.88 theta=15 freq=1.4 antdiam=25 nants=11 
basher - DEBUG : Command = obsrms jyperk=150 tsys=50 inttime=720.0 bw=20 coreta=0.88 theta=15 freq=1.4 antdiam=25 nants=11 
basher - DEBUG : 
OBSRMS: version 28-SEP-2007
 tsys  jyperk  freq   lambda    deltav     bw  inttime   nants    theta    coreta   rmsphase
   50.   9.4    1.40   214.14  4282.75   20.00  720.00    11.   15.0   15.0  0.88    0.
Rms Flux density:   0.546